In [19]:
# import re 
import pandas as pd
import re

In [20]:
 #importing files 

In [21]:
f = open('WhatsApp Chat with Priyanshu.txt', 'r', encoding = 'utf-8')

In [22]:
data = f.read()

In [23]:
%%capture
print(data)

In [24]:
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [25]:
%%capture
messages = re.split(pattern,data)[1:]
messages

In [26]:
dates = re.findall(pattern, data)

In [27]:
%%capture
dates

In [28]:
%%capture
df = pd.DataFrame({'user_message':messages,
                  'message_date':dates})
#convert message-date type
df['message_date'] = pd.to_datetime(df['message_date'], format = '%m/%d/%y, %H:%M - ')
df.rename(columns = {'message_date':'date'}, inplace= True)
df.head()

In [29]:
%%capture
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])
        
df['user'] = users
df[ 'message'] = messages
df.drop(columns=['user_message'], inplace = True)
df.head()

In [30]:
%%capture
df.shape

In [31]:
df['year'] = df['date'].dt.year

In [32]:
%%capture
df.head()

In [33]:
df['month'] = df['date'].dt.month_name()

In [34]:
df['day'] = df['date'].dt.day

In [35]:
df['hour'] = df['date'].dt.hour

In [36]:
df['minute'] = df['date'].dt.minute

In [37]:
%%capture
df.head()

In [38]:
%%capture
df[df['user'] == 'mehul'].shape

In [39]:
%%capture
words = []
for message in df['message']:
    words.extend(message.split())

In [40]:
%%capture
len(words)

In [41]:
pip install urlextract


Note: you may need to restart the kernel to use updated packages.


In [42]:
%%capture
from urlextract import URLExtract
extractor = URLExtract()
urls = extractor.find_urls("let's www.gmail.com have a url stackflow.com as an example of google.com and , http://facebook.com")
urls

In [43]:
%%capture
links = []
for message in df['message']:
    links.extend(extractor.find_urls(message))
    
links

In [44]:
%%capture
len(links)

In [45]:
%%capture
x = df['user'].value_counts().head()


In [46]:
%%capture
import matplotlib.pyplot as plt

In [47]:
%%capture
name = x.index
count = x.values
plt.bar(name,count)
plt.show

In [48]:
%%capture
round((df['user'].value_counts()/df.shape[0])*100,2)

In [49]:
%%capture
words = []
for message in df['message']:
    words.extend(message.split())
    
words

In [50]:
%%capture
pip install collections

In [51]:
%%capture
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [52]:
%%capture
temp = df[df['user'] == 'group_notification']
temp = temp[temp['message'] != '<Media omitted>\n']
temp

In [53]:
%%capture
f = open('stop_hinglish.txt', 'r')
stop_words = f.read().split()
stop_words

In [54]:
%%capture
words = []
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)
    
words

In [55]:
%%capture
import pandas as pd
import regex as re
import emoji
comments = df['message']
smiley_data = {'Smiley': [], 'Count': []}
pattern = r'([\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF])'

for comment in comments:
    smileys = re.findall(pattern, comment)
    for smiley in smileys:
        if smiley in smiley_data['Smiley']:
            index = smiley_data['Smiley'].index(smiley)
            smiley_data['Count'][index] += 1
        else:
            smiley_data['Smiley'].append(smiley)
            smiley_data['Count'].append(1)
smiley_df = pd.DataFrame(smiley_data)
smiley_df = smiley_df.sort_values(by='Count', ascending=False)
print(smiley_df.head())



In [56]:
%%capture
plt.bar(smiley_df['Smiley'].head(), smiley_df['Count'].head())

In [57]:
df['month_num'] = df['date'].dt.month

In [58]:
timeline = df.groupby(['year','month_num', 'month']).count()['message'].reset_index()

In [59]:
%%capture
timeline

In [60]:
time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + "-" + str(timeline['year'][i]))

In [61]:
timeline['time'] = time

In [62]:
%%capture
timeline

In [63]:
%%capture
plt.plot(timeline['time'],timeline['message'])
plt.xticks(rotation='vertical')
plt.show()

In [64]:
df['only_date']=df['date'].dt.date

In [65]:
daily_timeline = df.groupby('only_date').count()['message'].reset_index()

In [66]:
%%capture
plt.figure(figsize=(18,10))
plt.plot(daily_timeline['only_date'], daily_timeline['message'])

In [67]:
%%capture
df.head()

In [68]:
df['day_name'] = df['date'].dt.day_name()

In [69]:
%%capture
df['day_name'].value_counts()

In [70]:
%%capture
df['month'].value_counts()

In [71]:
%%capture
df.head()

In [72]:
%%capture
period = []
for hour in df[['day_name', 'hour']]['hour']:
    if hour ==23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour+1))
    else:
        period.append(str(hour) + "-" + str(hour+1))

In [73]:
df['period'] = period

In [74]:
%%capture
df.sample(5)

In [75]:
%%capture
import seaborn as sns
plt.figure(figsize=(20,6))
sns.heatmap(df.pivot_table(index='day_name', columns='period',values='message',aggfunc='count').fillna(0))
plt.yticks(rotation='horizontal')
plt.show()